In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import theano.tensor as tt
import pymc3 as pm
import theano
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


In [ ]:
X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
Xt = theano.shared(X_train)
yt = theano.shared(y_train)

with pm.Model() as iris_model:

    # Coefficients for features
    x = pm.Normal('x', mu=0, sigma=1, observed=)
    beta = pm.Normal('beta', 0, sigma=1e2, shape=(4, 3))
    # Transoform to unit interval
    a = pm.Flat('a', shape=(3,))
    p = tt.nnet.softmax(x.dot(beta) + a)
    
    observed = pm.Categorical('obs', p=p, observed=yt)

In [ ]:
with iris_model:

    # We'll use SVGD
    inference = pm.SVGD(n_particles=500, jitter=1)

    # Local reference to approximation
    approx = inference.approx

    # Here we need `more_replacements` to change train_set to test_set
    test_probs = approx.sample_node(p, more_replacements={Xt: X_test}, size=100)

    # For train set no more replacements needed
    train_probs = approx.sample_node(p)

In [ ]:
test_ok = tt.eq(test_probs.argmax(-1), y_test)
train_ok = tt.eq(train_probs.argmax(-1), y_train)
test_accuracy = test_ok.mean(-1)
train_accuracy = train_ok.mean(-1)

In [ ]:
eval_tracker = pm.callbacks.Tracker(
    test_accuracy=test_accuracy.eval,
    train_accuracy=train_accuracy.eval,
    test_p=test_probs.eval,
    train_p=train_probs.eval
)

In [ ]:
inference.fit(400, obj_optimizer=pm.adamax(learning_rate=0.1), callbacks=[eval_tracker]);

In [ ]:
test_data = pd.DataFrame(np.asarray(eval_tracker['test_accuracy']).T).melt()

# plt.plot(np.asarray(eval_tracker['train_accuracy']), color='blue')
sns.lineplot(x="variable", y="value", data=test_data, color='red')
plt.legend(['test_accuracy'])
plt.title('Training Progress')